In [ ]:
!apt-get install -y poppler-utils         # only for linux envorment

In [ ]:
!pip install easyocr

In [13]:
!pip install spacy 

In [19]:
!python -m spacy download en_core_web_sm

In [91]:
!pip install nltk

In [ ]:
!pip install pdf2image

In [2]:
import os
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
import easyocr
import cv2
import nltk
from matplotlib import pyplot as plt

# Path to PDF

In [27]:
PDF_PATH = 'txt.pdf'
OUTPUT_DIR = 'output_images'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [28]:
# Step 1: Try to extract text using pdfminer.six
extracted_text = extract_text(PDF_PATH)

In [29]:
if extracted_text.strip():  # Check if there is any extracted text
    print("Extracted Text from PDF:")
    print(extracted_text)
else:
    print("No text found, proceeding with OCR...")
    # Step 2: Convert PDF pages to images using pdf2image for OCR
    image_paths = []
    pages = convert_from_path(PDF_PATH, dpi=300)  # Converts each page of the PDF to an image at a resolution of 300 DPI (dots per inch)

    for i, page in enumerate(pages):
        image_path = os.path.join(OUTPUT_DIR, f'page_{i + 1}.jpeg')
        page.save(image_path, 'JPEG')  # Save each page as JPEG
        image_paths.append(image_path)

    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'])

Extracted Text from PDF:
Adnan Nazir

B.Tech , Electronics and Communication Engineering
National Institute of Technology (NIT), Srinagar
CGPA: 8.37 till 4th Sem

About Me

Theed Harwan, Srinagar,

Srinagar, J&K- 191121, India
Mob.: +91-6005605445
Email :adnanbhat70922@gmail.com
Github | Linkedin | Tableau | LeetCode

An enthusiastic student of Electronics and Communication Engineering and Data Science(self taught) who has experience in several
areas. Loves creative problem solving, getting exposure to multiple projects, and thriving in a collaborative environment.

Technical Skills

Python ,C++ , MySQL , Numpy , Pandas , Matplotlib , Seaborn , Scikit-learn ,TensorFlow Tableau, Power Bi , Kaggle, Object-Oriented
Programming , Data Structures and Algorithms ,Flask , Machine Learning , git & Github

Education

Degree/Certificate

Institute

CGPA

Year

B.Tech(ECE)

National Institute of Technology (NIT), Srinagar

8.37(Present)

2022 - Current

Higher Secondary

Govt. Boys higher seconda

# Process each image and extract text using OCR

In [52]:
final_detected_text = ""
if(extracted_text == ""):
    all_detected_text = []  # Store text from all pages
    for image_path in image_paths:
        # Read the image
        img = cv2.imread(image_path)
    
        # Preprocess the image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  # Binarization
        denoised = cv2.medianBlur(binary, 3)  # Noise removal using median blur
    
        # Perform OCR on the preprocessed image
        result = reader.readtext(denoised)
    
        detected_text = []  # Text for the current page
        font = cv2.FONT_HERSHEY_SIMPLEX
        spacer = 100
    
        # Draw boxes and extract text
        for detection in result:
            # Convert coordinates to integers
            top_left = tuple(map(int, detection[0][0]))
            bottom_right = tuple(map(int, detection[0][2]))
            text = detection[1]  # Extract detected text
    
            # Append detected text to the list
            detected_text.append(text)
    
            # Draw rectangle and put text
            img = cv2.rectangle(img, top_left, bottom_right, (0, 255, 0), 3)
            img = cv2.putText(img, text, (20, spacer), font, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
            spacer += 15
    
        # Save detected text for the current page
        all_detected_text.append("\n".join(detected_text))
    
        # Display the image with detected text
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Page {image_paths.index(image_path) + 1}")
        plt.show()
        # all_detected_text.append("\n".join(detected_text))
        final_detected_text = "".join(all_detected_text)
else :
    final_detected_text = "".join(extracted_text)

# Combine all detected text into a single string

In [44]:
# all_detected_text.append("\n".join(detected_text))
# final_detected_text = "\n\n".join(all_detected_text)

# Print the final detected text

In [53]:
print("Final Detected Text from OCR:")
print(final_detected_text)

Final Detected Text from OCR:
Adnan Nazir

B.Tech , Electronics and Communication Engineering
National Institute of Technology (NIT), Srinagar
CGPA: 8.37 till 4th Sem

About Me

Theed Harwan, Srinagar,

Srinagar, J&K- 191121, India
Mob.: +91-6005605445
Email :adnanbhat70922@gmail.com
Github | Linkedin | Tableau | LeetCode

An enthusiastic student of Electronics and Communication Engineering and Data Science(self taught) who has experience in several
areas. Loves creative problem solving, getting exposure to multiple projects, and thriving in a collaborative environment.

Technical Skills

Python ,C++ , MySQL , Numpy , Pandas , Matplotlib , Seaborn , Scikit-learn ,TensorFlow Tableau, Power Bi , Kaggle, Object-Oriented
Programming , Data Structures and Algorithms ,Flask , Machine Learning , git & Github

Education

Degree/Certificate

Institute

CGPA

Year

B.Tech(ECE)

National Institute of Technology (NIT), Srinagar

8.37(Present)

2022 - Current

Higher Secondary

Govt. Boys higher se

# Now extraction important details 

In [54]:
import spacy 

nlp = spacy.load("en_core_web_sm")
skill_path = 'skills.jsonl'

In [55]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [56]:
doc = nlp("My name is Adnan Nazir and i am working on NLP and python project")
doc.ents

(Adnan Nazir, NLP, python)

# clean our data

In [81]:
from spacy.lang.en.stop_words import STOP_WORDS
import re 

def preprocessing(sentence):
    stopwords = list(STOP_WORDS)
     # Remove unwanted characters: ., -, |, •, etc.
    sentence = re.sub(r"[.,\-|•]", " ", sentence)
    # Remove extra whitespace
    sentence = re.sub(r"\s+", " ", sentence).strip()
    # Tokenize and process with spaCy
    doc = nlp(sentence)
    clean_tokens = []

    for token in doc:
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SYM' and \
           token.pos_ != 'SPACE':
            clean_tokens.append(token.lemma_.lower().strip())
    
    return " ".join(clean_tokens)

In [82]:
preprocess_text = preprocessing(final_detected_text)
preprocess_text

'adnan nazir b tech electronics communication engineering national institute technology nit srinagar cgpa 8 37 till 4th sem about i theed harwan srinagar srinagar j&k 191121 india mob +91 6005605445 email adnanbhat70922@gmail com github linkedin tableau leetcode an enthusiastic student electronics communication engineering data science(self teach experience area love creative problem solve get exposure multiple project thrive collaborative environment technical skills python c++ mysql numpy pandas matplotlib seaborn scikit learn tensorflow tableau power bi kaggle object oriented programming data structures algorithms flask machine learning git & github education degree certificate institute cgpa year b tech(ece national institute technology nit srinagar 8 37(present 2022 current high secondary govt boys high secondary school shalimar srinagar secondary fams school harwan srinagar 9 4 88 4 % 2020 2021 2018 2019 projects hand sign classification model svm cnn link build hand sign recogni

# Extracting skills

In [83]:
def get_skills(text):
  doc = nlp(text)

  skills = []

  for ent in doc.ents:
    if ent.label_ == 'SKILL':
      skills.append(ent.text)

  return list(set(skills))

In [84]:
op_skills = get_skills(preprocess_text)
op_skills

['data visualization',
 'computer vision',
 'github',
 'data structures',
 'mysql',
 'python',
 'numpy',
 'rest api',
 'c++',
 'technical skills',
 'tableau',
 'api development',
 'pandas',
 'engineering',
 'svm',
 'certificate',
 'algorithms',
 'feature extraction',
 'machine learning',
 'certification',
 'flask',
 'data science',
 'power bi',
 'tensorflow',
 'scikit learn']

# Extract adjectives and adverbs

In [92]:
import nltk
from nltk.corpus import words
nltk.download('words')

[nltk_data] Downloading package words to C:\Users\Adnan
[nltk_data]     Nazir\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [93]:
valid_words = set(words.words())
def get_adjectives_and_adverbs(text):
    doc = nlp(text)
    
    # Extract adjectives
    adjectives = [
        token.lemma_.lower() 
        for token in doc 
        if token.pos_ == "ADJ" and token.lemma_.lower() in valid_words
    ]
    
    # Extract adverbs
    adverbs = [
        token.lemma_.lower() 
        for token in doc 
        if token.pos_ == "ADV" and token.lemma_.lower() in valid_words
    ]
    return adjectives, adverbs

In [94]:
adjectives, adverbs = get_adjectives_and_adverbs(preprocess_text)

In [95]:
adjectives

['enthusiastic',
 'creative',
 'multiple',
 'collaborative',
 'technical',
 'current',
 'high',
 'secondary',
 'high',
 'secondary',
 'secondary',
 'precise',
 'lightweight',
 'high',
 'robust',
 'comprehensive',
 'specific',
 'seamless',
 'interactive',
 'comprehensive',
 'specific',
 'percentile']

In [87]:
adverbs

[]

# visualize

In [71]:
from spacy import displacy

In [72]:
doc = nlp(preprocess_text)

In [73]:
displacy.render(doc , style = 'ent')